<a href="https://colab.research.google.com/github/MartinSantaGitHub/python-ml-course/blob/master/code/T11%20-%201%20-%20TensorFlow101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introducción a Tensor Flow**

In [64]:
%tensorflow_version 1.x

In [65]:
import tensorflow as tf

In [66]:
print(tf.__version__)

1.15.2


In [67]:
x1 = tf.constant([1,2,3,4,5])
x2 = tf.constant([6,7,8,9,10])

In [68]:
res = tf.multiply(x1,x2)
res

<tf.Tensor 'Mul_1:0' shape=(5,) dtype=int32>

In [69]:
print(res)

Tensor("Mul_1:0", shape=(5,), dtype=int32)


In [70]:
res.dtype

tf.int32

In [71]:
sess = tf.Session()
print(sess.run(res))
sess.close()

[ 6 14 24 36 50]


In [72]:
with tf.Session() as sess:
  output = sess.run(res)
  print(output)

[ 6 14 24 36 50]


In [73]:
config = tf.ConfigProto(log_device_placement = True)
config = tf.ConfigProto(allow_soft_placement = True)